# Janome
**Janome** は Python 言語で作成された形態素解析器です。MeCab とは異なり、別のソフトウェアのインストールを前提としません。ただし、解析対象の文章（テキスト）のサイズが大きくなると、解析に時間がかかることが欠点となります。大規模なテキスト群を対象とする場合は、MeCab を使うほうがストレスがありません。


Anaconda Prompt を起動します。Amaconda のインストール時にオプションとして Just Me ではなくシステム全体へのインストールを行った場合は、右クリック「その他->管理者として起動」として起動します。以下のように打ち込み、janomeをインストールします。

> conda install -c conda-forge janome 

上のコマンドが機能しない場合は、下のようにします。
 
> pip install janome

Jupyterのセルからも実行できます。


In [2]:
!pip install janome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 5.8 MB/s eta 0:00:00m eta 0:00:010:01:01


インストールしたら、Janome を試してみましょう。



In [3]:
from janome.tokenizer import Tokenizer

t = Tokenizer()
toks = t.tokenize('すもももももももものうち')

In [4]:
for tok in toks:
    print(tok)

すもも	名詞,一般,*,*,*,*,すもも,スモモ,スモモ
も	助詞,係助詞,*,*,*,*,も,モ,モ
もも	名詞,一般,*,*,*,*,もも,モモ,モモ
も	助詞,係助詞,*,*,*,*,も,モ,モ
もも	名詞,一般,*,*,*,*,もも,モモ,モモ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
うち	名詞,非自立,副詞可能,*,*,*,うち,ウチ,ウチ


In [2]:
toks = t.tokenize('ご飯を食べた')
for tok in toks:
    print(tok)

ご飯	名詞,一般,*,*,*,*,ご飯,ゴハン,ゴハン
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
食べ	動詞,自立,*,*,一段,連用形,食べる,タベ,タベ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ





## Janome の解析結果確認


`tokenize()` は形態素解析の結果をリストとして返しますが、リストの要素それぞれに `surface` （表層形）、 `part_of_speech`（品詞）、 `infl_type` （活用型1）、`infl_form`（活用形2）、 `base_form`（基本形、見出し語）、 `reading`（読み）、 `phonetic`（発音）という要素があります。


In [5]:
toks = t.tokenize('西郷隆盛はご飯を食べた。')
for tok in toks:
    print('表層形：' + tok.surface)
    print('品詞情報：'+ tok.part_of_speech)
    print('活用形１：'+ tok.infl_type)
    print('活用形２：'+ tok.infl_form)
    print('基本形：'+ tok.base_form)
    print('読み：'+ tok.reading)
    print('---------------------')

表層形：西郷
品詞情報：名詞,固有名詞,人名,姓
活用形１：*
活用形２：*
基本形：西郷
読み：サイゴウ
---------------------
表層形：隆盛
品詞情報：名詞,固有名詞,人名,名
活用形１：*
活用形２：*
基本形：隆盛
読み：タカモリ
---------------------
表層形：は
品詞情報：助詞,係助詞,*,*
活用形１：*
活用形２：*
基本形：は
読み：ハ
---------------------
表層形：ご飯
品詞情報：名詞,一般,*,*
活用形１：*
活用形２：*
基本形：ご飯
読み：ゴハン
---------------------
表層形：を
品詞情報：助詞,格助詞,一般,*
活用形１：*
活用形２：*
基本形：を
読み：ヲ
---------------------
表層形：食べ
品詞情報：動詞,自立,*,*
活用形１：一段
活用形２：連用形
基本形：食べる
読み：タベ
---------------------
表層形：た
品詞情報：助動詞,*,*,*
活用形１：特殊・タ
活用形２：基本形
基本形：た
読み：タ
---------------------
表層形：。
品詞情報：記号,句点,*,*
活用形１：*
活用形２：*
基本形：。
読み：。
---------------------


## ファイルからの読み込み
さて、上では `tokenzie()` に日本語の文章を直接指定しましたが、一般には、別に用意されたファイルから文章を読み込むのが普通です。まずファイルを用意します。
次のように書かれたファイル short.txt を data フォルダに用意します。

> 国境の長いトンネルを抜けると雪国であった。

Windows ではメモ帳を使うことができますが、保存の際、文字コードとして `UTF-8' を選んでください。このファイルを、以下で表示されるフォルダに保存します。


In [6]:
import os
os.getcwd()

'/mnt/2bddf92b-47f9-4809-95a5-b91e7f25af27/myData/GitHub/python_de_textmining'

In [7]:
## テキストファイルの読み取り
f = open('data/short.txt', 'r', encoding='utf-8')
text = f.read()

t = Tokenizer()

for token in t.tokenize(text):
    print(token.surface)

## テキストファイルを閉じる
f.close()

国境
の
長い
トンネル
を
抜ける
と
雪国
で
あっ
た
。



さて、少し複雑な処理を試してみましょう。テキスト分析では、形態素を大きく内容語と機能語に分けて考えます。内容語は、テキストのテーマを端的に表す形態素、機能語は文法的に重要な形態素です。助詞の「は」や「が」などが代表ですが、出現頻度から書き手の推定に有効だと考えられています。



In [9]:
f = open('data/short.txt', 'r', encoding='utf-8')
text = f.read()

t = Tokenizer()

## janomeの形態素解析から抜き出す名詞リスト
meishi_lst = []

for token in t.tokenize(text):
    ## 品詞が名詞なら
    if '名詞' in token.part_of_speech:
        ## 表層形(語句)を出力
        meishi = token.surface
        ## 名詞リストに追加
        meishi_lst.append(meishi)

## テキストファイルを閉じる
f.close()

meishi_lst

['国境', 'トンネル', '雪国']

## 青空文庫

さて、ここで本格的なテキスト処理に挑戦してみましょう。青空文庫からテキストを取り出して、形態素解析にかけるという処理を行ってみます。その上で、固有名詞を取り出してみます。


In [10]:
from AozoraDL import aozora
## URL を文字列として指定
aozora('https://www.aozora.gr.jp/cards/000035/files/1567_ruby_4948.zip')

Download URL
URL: https://www.aozora.gr.jp/cards/000035/files/1567_ruby_4948.zip
1567_ruby_4948/hashire_merosu.txt
ファイルの作成：hashire_merosu.txt


In [11]:
f = open('hashire_merosu.txt', 'r')
text = f.read()
t = Tokenizer()

## janomeの形態素解析から抜き出す名詞リスト
meishi_lst = []

for token in t.tokenize(text):
    ## 品詞細分類が固有名詞なら
    if "固有名詞" in token.part_of_speech:
        ## 表層形(語句)を出力
        meishi = token.surface
        ## 名詞リストに追加
        meishi_lst.append(meishi)

## テキストファイルを閉じる
f.close()
## リストを表示
print(meishi_lst)

['王', '王', 'セリヌンティウス', '王', 'えい', 'えい', '二里', '三里', '猛', '木葉', '渡', 'ゼウス', '韋駄天', '潺々', '清水', 'ゼウス', '小川', 'はるか', 'メロス', '勇', 'ちか', 'メロス', 'メロス']



## 自作関数のモジュール化

さて、形態素解析の結果を使って分析を行う場合、利用する品詞情報を指定したくなります。具体的には、名詞や動詞、形容詞のみを抽出してデータとしたいことがあります。先にみたように、形態素ごとの品詞情報は、形態素ごとに `part_of_speech` として保存されています。そこで、これが名詞か動詞、あるいは形容詞だった場合だけ形態素をデータとして取り出すことを考えます。本書ではこの処理を繰り返し行いますので、これを関数として定義してしまいましょう。また、関数定義を別ファイルとして保存して、これをモジュールとして読み込む方法を紹介します。



In [ ]:
# -*- coding: utf-8 -*-

from janome.tokenizer import Tokenizer

t = Tokenizer()

def tokens(text, pos = ['名詞','形容詞','動詞']):
    word_list = []
    for token in t.tokenize(text):
    ## 品詞が名詞なら
        tp = (token.part_of_speech).split(',')
        if tp[0] in pos:
            word_list.append(token.base_form)
    return word_list

if __name__ == '__main__':
    out = tokens('これは良い本です。')
    print(out)



ちなみにターミナルを起動して、以下のように実行することもできます。

```
$ python my_janome.py 
['これ', '良い', '本']
```


In [12]:
import my_janome
print('デフォルトでは名詞、形容詞、助詞を出力')
out = my_janome.tokens('ランチを食べました。')
print(out)
print('助詞のみ抽出')
out = my_janome.tokens("ランチを食べました。", pos = '助詞')
print(out)

デフォルトでは名詞、形容詞、助詞を出力
['ランチ', '食べる']
助詞のみ抽出
['を']


In [ ]:
# -*- coding: utf-8 -*-

## my_janome.py 

from janome.tokenizer import Tokenizer

t = Tokenizer()

def tokens(text, pos = ['名詞','形容詞','動詞']):
    word_list = []
    for token in t.tokenize(text):
    ## 品詞が名詞なら
        tp = (token.part_of_speech).split(',')
        if len(pos) < 1 or tp[0] in pos:
            word_list.append(token.base_form)
    return word_list

if __name__ == '__main__':
    out = tokens('これは良い本です。')
    print(out)

## ストップワード


たとえば、デフォルトの Janome で「今日は本を読んで過ごした。」を形態素解析してみると、以下のような結果になります。



In [13]:
from janome.tokenizer import Tokenizer
t = Tokenizer()
toks = t.tokenize('今日はこの本を読みます。')
for tok in toks:
    print(tok)

今日	名詞,副詞可能,*,*,*,*,今日,キョウ,キョー
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
この	連体詞,*,*,*,*,*,この,コノ,コノ
本	名詞,一般,*,*,*,*,本,ホン,ホン
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
読み	動詞,自立,*,*,五段・マ行,連用形,読む,ヨミ,ヨミ
ます	助動詞,*,*,*,特殊・マス,基本形,ます,マス,マス
。	記号,句点,*,*,*,*,。,。,。



Janome の解析結果では、`part_of_speech` を参照すれば、必要な品詞情報だけを取り出すことができるようになります。作成したばかりのモジュールを使って、次の文章を解析してみましょう。


In [14]:
import my_janome
out = my_janome.tokens('これは良い本だ。')
print(out)

['これ', '良い', '本']


「これ」が名詞として抽出されていますが、内容語とはいえないでしょう。ちなみに、「これ」の品詞細分類を確認してみましょう。



In [15]:
from janome.tokenizer import Tokenizer
t = Tokenizer()
tokens = t.tokenize('これは良い本だ。')
for tok in tokens:
    if tok.surface == 'これ':
        print(tok.part_of_speech)

名詞,代名詞,一般,*



機能語と判断される形態素をあらかじめリストにしておいて、これと照合することで不要語を一気に削除してしまう方法もあります。こうしたリストをストップワードと言います。分析目的にあわせてストップワードは自身を作成するのがベストですが、公開されているリストを利用することもできます。こうしたリストとして、京都大学情報学研究科社会情報学専攻田中克己研究室が公開している SlothLib [^SlothLib] がありますので、ここで利用させてもらいましょう。

[^SlothLib]:  http://www.dl.kuis.kyoto-u.ac.jp/slothlib/


ここでは stopword リストを stopwords.txt として、data フォルダに保存します。


In [17]:
import urllib.request
url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
urllib.request.urlretrieve(url, 'data/stopwords.txt')
stopwords = []
with open('data/stopwords.txt', 'r', encoding='utf-8') as f:
	stopwords = [w.strip() for w in f]

In [18]:
from janome.tokenizer import Tokenizer
t = Tokenizer()
tokens = t.tokenize('これは良い本です。')
pos = ['名詞','形容詞','動詞']
word_list = []
for token in tokens:
    tp = (token.part_of_speech).split(',')
    if token.base_form not in stopwords:
        if tp[0] in pos :
            word_list.append(token.base_form)
print(word_list)

['良い', '本']


In [ ]:
# -*- coding: utf-8 -*-

##  my_janome_stopwords.py
from janome.tokenizer import Tokenizer

t = Tokenizer()

def tokens(text, pos = ['名詞','形容詞','動詞'] , stopwords_list=[]):
    word_list = []
    for token in t.tokenize(text):
    ## 品詞が名詞なら
        tp = (token.part_of_speech).split(',')
        if token.base_form not in stopwords_list:
            if len(pos) < 1 or (tp[0] in pos):
                word_list.append(token.base_form)
    return word_list

if __name__ == '__main__':
    out = tokens('これは良い本です。')
    print(out)

In [19]:
import my_janome_stopwords as jnm
out = jnm.tokens('これは良い本です。')
print(out)

['これ', '良い', '本']


In [20]:
import my_janome_stopwords as jnm
out = jnm.tokens('これは良い本です。', stopwords_list=stopwords)
print(out)

['良い', '本']


本書の分析例では形態素解析器として MeCab を利用したコードを掲載しますが、MeCab をインポートする行を `import my_janome_stopwords as jnm` と変えることで、同様の出力を得ることができます（多少、異なる結果になることもあります）。
ただし、MeCab に比べると Janome の解析速度は非常に遅いので、入力データ（日本語文章）が大きくなると、結果が得られるまでかなり時間がかかることがあります。

